In [1]:
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

In [2]:
service = Service('/usr/local/bin/chromedriver')

In [3]:
def scroll(driver, timeout, max_scrolls):
    scroll_pause_time = timeout
    
    current_scroll = 0

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(scroll_pause_time)

#         # Calculate new scroll height and compare with last scroll height
#         new_height = driver.execute_script("return document.body.scrollHeight")
#         if new_height == last_height:
#             # If heights are the same it will exit the function
#             break
#         last_height = new_height
        
        current_scroll += 1
        if current_scroll >= max_scrolls:
            break

In [9]:
def get_vivino_explore_source(max_scrolls=50):
    service.start()
    driver = webdriver.Remote(service.service_url)
    driver.get('https://www.vivino.com/explore')
    
    scroll(driver, 2, max_scrolls)
    
    page_source = driver.page_source
    
    driver.quit()

    return page_source

In [5]:
def get_wine_links(source):
    links = []
    
    explore_soup = BeautifulSoup(source, "html.parser")
    cards = explore_soup.find_all("div", class_="explorerCard__explorerCard--3Q7_0")
    
    for card in cards:
        full_link = "https://vivino.com" + card.a['href']
        links.append(full_link[:-32])
    
    return links

In [47]:
def save_links_to_file(path, links):
    with open(path, "w") as f:
        for link in f:
            f.write(link+"\n")

In [13]:
vivino_explore_source = get_vivino_explore_source(50)

In [14]:
wine_links = get_wine_links(vivino_explore_source)
print(len(wine_links))
wine_links[:5]

1725


['https://vivino.com/belle-glos-dairyman-vineyard-pinot-noir/w/1561411?year=2019&price_id=23873673',
 'https://vivino.com/alto-moncayo-garnacha/w/98782?year=2017&price_id=20475900',
 'https://vivino.com/gitana-lupi-rezerva/w/1468452?year=2016&price_id=23006244',
 'https://vivino.com/merry-edwards-wines-sauvignon-blanc/w/12266?year=2018&price_id=20310937',
 'https://vivino.com/altaneve-prosecco-superiore/w/2052330?year=N.V.&price_id=9768113']

In [46]:
save_links_to_file("datasets/vivino_explore_links.txt", wine_links)